# 直接推理

In [ ]:
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks
text_generation_zh = pipeline(Tasks.text_generation, model='outputs/Lgpt/output/')
result_zh = text_generation_zh("这是一个测试")
print(result_zh['text'])

# 前处理

In [4]:
from modelscope.hub.snapshot_download import snapshot_download
from modelscope.preprocessors import TextGenerationPreprocessor

model_id = 'damo/nlp_gpt3_text-generation_chinese-large'
model_dir = snapshot_download(model_id)
sentence = '这是一个测试文本'

preprocessor = TextGenerationPreprocessor(model_dir, sequence_length=500)
result = preprocessor(sentence)
print(result, result['input_ids'].shape)

2022-11-27 12:36:22,225 - modelscope - INFO - Model revision not specified, use the latest revision: v1.0.0
2022-11-27 12:36:22,396 - modelscope - INFO - File config.json already in cache, skip downloading!
2022-11-27 12:36:22,397 - modelscope - INFO - File configuration.json already in cache, skip downloading!
2022-11-27 12:36:22,398 - modelscope - INFO - File gpt.png already in cache, skip downloading!
2022-11-27 12:36:22,400 - modelscope - INFO - File pytorch_model.bin already in cache, skip downloading!
2022-11-27 12:36:22,401 - modelscope - INFO - File README.md already in cache, skip downloading!
2022-11-27 12:36:22,401 - modelscope - INFO - File vocab.txt already in cache, skip downloading!
2022-11-27 12:36:22,423 - modelscope - INFO - The key of sentence1: None, The key of sentence2: None, The key of label: label
2022-11-27 12:36:22,424 - modelscope - WARNING - [Important] first_sequence attribute is not set, this will cause an error if your input is a dict.


{'input_ids': tensor([[    2, 17059,  6393,   202,   244,  8213, 16031,  6225,  6646,     3,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

### 用tokenizer，效果一样的
- 这里gpt会有特殊token cls和sep

In [3]:
from transformers import BertTokenizer
# model_dir 为模型配置文件路径，需包含 vocab.txt 文件
tokenizer = BertTokenizer.from_pretrained('nlp_gpt3_text-generation_chinese-large')
output = tokenizer('这是一个测试文本', return_tensors='pt')
print(output)
# 将tokenizer的输出传入模型

{'input_ids': tensor([[    2, 17059,  6393,   202,   244,  8213, 16031,  6225,  6646,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


# 前向传播与生成
1.   传的是个字典，然后传labels就会计算loss，不然就是None。返回值为(loss, logits)
- labels可以传-100，交叉熵会忽略，这是nn.CrossEntropyLoss自带的
2.  训练过程可以看情况把labels调整忽略不计算loss
3. huggingface官方默认是把输入当作labels（去掉第一个字），logits会去掉最后一个字符(here)[https://huggingface.co/transformers/v4.6.0/_modules/transformers/models/gpt2/modeling_gpt2.html#GPT2LMHeadModel]

In [4]:
from modelscope.models.nlp import PalmForTextGeneration
from modelscope.models.nlp import GPT3ForTextGeneration
from modelscope.models import Model
model = GPT3ForTextGeneration.from_pretrained('nlp_gpt3_text-generation_chinese-large')
model(result)['logits'].shape

2022-11-26 21:59:45,470 - modelscope - INFO - initialize model from nlp_gpt3_text-generation_chinese-large


torch.Size([1, 128, 25600])

- generate只支持一句的，本质是只取了第一句，然后去掉[sep]特殊符号，调用huggingface的generate。
- 返回值是id，要用tokenizer.decode解码
- 这里默认是do_sample的，所以每次生成不一样，具体看源码

In [5]:
model.eval()
result['max_length'] = 30
seq = model.generate(result)
seq['sequences']

tensor([    2, 17059,  6393,   202,   244,  8213, 16031,  6225,  6646, 21124,
         6393,   202,   244,  2546, 12681, 10574,  8213, 16031, 21124,  6393,
          202,   244,  1721,   431,  3779,  6225,  6646, 17061, 15126,  1386])

In [6]:
tokenizer.decode(seq['sequences'], skip_special_tokens=True)

'这 是 一 个 测 试 文 本 ， 是 一 个 在 线 的 测 试 ， 是 一 个 可 以 对 文 本 进 行 加'

In [78]:
import modelscope.models
modelscope.models.__file__

'/home/weixiang/.miniconda3/envs/modelscope/lib/python3.7/site-packages/modelscope/models/__init__.py'

In [122]:
model.model.config

GPT3Config {
  "apply_query_key_layer_scaling": true,
  "apply_residual_connection_post_layernorm": false,
  "attention_dropout": 0.1,
  "attention_probs_dropout_prob": 0.1,
  "attention_softmax_in_fp32": false,
  "attention_type": "self",
  "bf16": false,
  "bias_dropout_fusion": true,
  "bias_gelu_fusion": true,
  "eod_id": 7,
  "ffn_hidden_size": 4096,
  "fp16": false,
  "fp32_residual_connection": false,
  "hidden_act": "gelu",
  "hidden_dropout": 0.1,
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "init_method_std": 0.02,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "kv_channels": 64,
  "layer_norm_eps": 1e-12,
  "layernorm_epsilon": 1e-12,
  "masked_softmax_fusion": true,
  "max_position_embeddings": 2048,
  "model_type": "gpt3",
  "no_persist_layer_norm": false,
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "sequence_parallel": false,
  "tokens_to_generate": 100,
  "top_k": 0,
  "top_p": 0.9,
  "transformers_version": "4.24.0",
  "type_vocab_si

# tranier训练测试
- 内部模型用的也是GPT3ForTextGeneration

In [ ]:
from modelscope.msdatasets import MsDataset
from modelscope.trainers import build_trainer
from modelscope.msdatasets import MsDataset
from modelscope.utils.hub import read_config
from modelscope.metainfo import Metrics, Trainers
from datasets import Dataset
from modelscope.msdatasets import MsDataset

# 模拟训练数据集
src_dataset_dict = {
    'src_txt': [
        '你好啊', '中国是第一名', '我是你的父亲啊'
    ]
}
train_dataset = MsDataset(Dataset.from_dict(src_dataset_dict))

print (train_dataset)
max_epochs = 1
tmp_dir = "./gpt3_poetry"

num_warmup_steps = 100
def noam_lambda(current_step: int):
    current_step += 1
    return min(current_step ** (-0.5), current_step * num_warmup_steps ** (-1.5))

def cfg_modify_fn(cfg):
    cfg.train.lr_scheduler = {
        "type": "LambdaLR",
        "lr_lambda": noam_lambda,
        "options": {"by_epoch": False}
    }
    cfg.train.optimizer = {
        "type": "AdamW",
        "lr": 1e-4
    }
    cfg.train.dataloader = {"batch_size_per_gpu": 16, "workers_per_gpu": 1}
    return cfg

kwargs = dict(
    model='damo/nlp_gpt3_text-generation_chinese-large',
    train_dataset=train_dataset,
    max_epochs=max_epochs,
    work_dir=tmp_dir,
    cfg_modify_fn=cfg_modify_fn)

# 构造 trainer 并进行训练
trainer = build_trainer(
    name=Trainers.nlp_base_trainer, default_args=kwargs)
trainer.train()

In [166]:
trainer.model = trainer.model.cpu()
tokenizer.decode(trainer.model.generate(result)['sequences'])

'[CLS] 这 是 一 个 测 试 文 本 的 网 站 ， 你 可 以 用 它 来 帮 助 自 己 在 你 的 网 站 上 发 布 一 个 文 件 。 你 可 以 通 过 这 种 网 站 ， 来 帮 助 自 己 在 你 的 网 站 上 发 布 一 个 文 件 。 这 是 一 个 很 棒 的 网 站 ， 你 可 以 用 这 个 网 站 ， 来 帮 助 自 己 在 你 的 网 站 上 发 布 一 个 文 件 。 这 是 一 个 很 棒 的 网 站 ， 你 可 以 用 它 来 帮 助 自 己 在 你 的 网 站 上 发'